In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('DEV_OPENAI_API_KEY')
llm = ChatOpenAI(model = "gpt-4o", openai_api_key=api_key,temperature=0,max_tokens=4096,)

llm.invoke([
    HumanMessage("잘 지냈어?")
])

In [ ]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool
def get_current_time(timezone : str,location : str):
    tz = pytz.timezone(timezone)
    now = datetime.now(tz=tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now}'
    print(location_and_local_time)
    return location_and_local_time

In [ ]:
tools = [get_current_time]
tool_dict = {"get_current_time": get_current_time}

llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain_core.messages import HumanMessage,SystemMessage

messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇 시야?")
]

response = llm_with_tools.invoke(messages)
messages.append(response)

print(response)

In [ ]:
for tool_call in response.tool_calls :
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    
print(messages) 

In [ ]:
llm_with_tools.invoke(messages)

In [ ]:
from pydantic import BaseModel , Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예 : AAPL)")
    period : str = Field(...,title="기간", description="(주식 데이터 조회 기간 (예 : 1d, 1mo,1y))")

In [ ]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input : StockHistoryInput) -> str:
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period = stock_history_input.period)
    history_md = history.to_markdown()
    
    return history_md

tools = [get_yf_stock_history, get_current_time]
tool_dict = {"get_yf_stock_history": get_yf_stock_history, "get_current_time": get_current_time}

llm_with_tools = llm.bind_tools(tools)

In [ ]:
messages.append(HumanMessage("테슬라는 한 달 전에 비해 주가가 얼마나 내렸나"))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

In [ ]:
for tool_call in response.tool_calls :
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

In [ ]:
llm_with_tools.invoke(messages)

In [ ]:
for c in llm.stream([HumanMessage("잘 지냈어? 한국 사회의 문제점이 무엇인지 이야기해줘.")]):
    print(c.content,end=' | ')

In [ ]:
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?")
]

response = llm_with_tools.stream(messages)

is_first = True
for chunk in response:
    print("chunk type:", type(chunk))
    
    if is_first:
        is_first = False
        gathered = chunk
    else : 
        gathered += chunk
        
    print("content : ",gathered.content, "tool_call_chunk",gathered.tool_calls)
    
messages.append(gathered)
gathered

In [ ]:
for tool_call in gathered.tool_calls :
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    
messages

In [ ]:
for c in llm_with_tools.stream(messages):
    print(c.content,end=' | ')